In [76]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [12]:
data=pd.read_csv("D:\\anime.csv")

In [14]:
data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [16]:
#Data Preprocessing & Feature Extraction
data.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [18]:
data.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [20]:
data["genre"].mode()

0    Hentai
Name: genre, dtype: object

In [22]:
data["genre"]=data["genre"].fillna(data["genre"].mode()[0])

In [24]:
data["type"]=data["type"].fillna(data["type"].mode()[0])
data["rating"]=data["rating"].fillna(0)
data.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [26]:
data1=pd.pivot_table(data=data,index='anime_id',columns='genre',values='rating')
data1.fillna(0,inplace=True)
data1

genre,Action,"Action, Adventure","Action, Adventure, Cars, Comedy, Sci-Fi, Shounen","Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports","Action, Adventure, Cars, Sci-Fi","Action, Adventure, Comedy","Action, Adventure, Comedy, Demons, Drama, Ecchi, Horror, Mystery, Romance, Sci-Fi","Action, Adventure, Comedy, Demons, Fantasy, Magic","Action, Adventure, Comedy, Demons, Fantasy, Magic, Romance, Shounen, Supernatural","Action, Adventure, Comedy, Demons, Fantasy, Martial Arts, Shounen, Super Power",...,Slice of Life,"Slice of Life, Space","Slice of Life, Supernatural",Space,Sports,"Super Power, Supernatural, Vampire",Supernatural,Thriller,Vampire,Yaoi
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#Recommendation System & Evaluation
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack

In [63]:
# tf-idf for genre
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(data["genre"])

# scale numerical features
scaler = MinMaxScaler()
num_features = scaler.fit_transform(data[["rating", "members"]])

# combine genre + rating + members
from scipy.sparse import csr_matrix
num_features_sparse = csr_matrix(num_features)
features = hstack([tfidf_matrix, num_features_sparse])

print("Final feature matrix shape:", features.shape)

Final feature matrix shape: (12294, 48)


In [95]:
#Cosine Similarity
cosine_sim = cosine_similarity(features, features)
print("Cosine similarity matrix shape:", cosine_sim.shape)

Cosine similarity matrix shape: (12294, 12294)


In [91]:
#Recommendation Function
# map anime name to index
indices = pd.Series(data.index, index=data["name"]).drop_duplicates()

def recommend(anime_name, top_n=5, threshold=0.3):
    if anime_name not in indices:
        return f"'{anime_name}' not found"
    idx = indices[anime_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = [(i, score) for i, score in sim_scores if score >= threshold and i != idx]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_indices = [i for i, _ in sim_scores[:top_n]]
    return data.iloc[sim_indices][["name", "genre", "type", "rating"]]

In [85]:
# Genre-only similarity
cosine_sim_genre = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hybrid similarity (genre + rating + members)
cosine_sim_hybrid = cosine_similarity(features, features)

def recommend_simple(title, cosine_sim, topn=5):
    idx = data.index[data['name'] == title][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:topn+1]
    return data['name'].iloc[[i[0] for i in sim_scores]]

In [87]:
# Example comparison
print("Genre-only recommendations:")
print(recommend_simple("Steins;Gate", cosine_sim_genre))

print("\nHybrid recommendations:")
print(recommend_simple("Steins;Gate", cosine_sim_hybrid))

Genre-only recommendations:
59              Steins;Gate Movie: Fuka Ryouiki no Déjà vu
126                  Steins;Gate: Oukoubakko no Poriomania
196      Steins;Gate: Kyoukaimenjou no Missing Link - D...
10898                                        Steins;Gate 0
5126                                         Under the Dog
Name: name, dtype: object

Hybrid recommendations:
59             Steins;Gate Movie: Fuka Ryouiki no Déjà vu
126                 Steins;Gate: Oukoubakko no Poriomania
196     Steins;Gate: Kyoukaimenjou no Missing Link - D...
5126                                        Under the Dog
493                                       Higashi no Eden
Name: name, dtype: object


In [105]:
#Evaluation Scores (Precision, Recall, F1)
sample_titles = ["Steins;Gate", "Fullmetal Alchemist: Brotherhood", "Kimi no Na wa."]
y_true = []
y_pred = []

for title in sample_titles:
    recs = recommend(title)
    target_genres = set(data.loc[data["name"] == title, "genre"].values[0].split(', '))
    for _, row in recs.iterrows():
        rec_genres = set(row["genre"].split(', '))
        overlap = len(target_genres & rec_genres) > 0
        y_true.append(1)
        y_pred.append(1 if overlap else 0)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

In [107]:
print("\nEvaluation Scores:")
print("Precision:", round(precision, 2))
print("Recall:", round(recall, 2))
print("F1-score:", round(f1, 2))


Evaluation Scores:
Precision: 1.0
Recall: 1.0
F1-score: 1.0


**Here we calculate how good our system is at recommending anime with similar genres. Higher precision means recommendations are relevant, higher recall means we did not miss many relevant ones, and F1 balances both.**

**Results and Analysis**

**The system successfully recommends anime titles that share genres and similar popularity (rating and members). By changing the similarity threshold, we can control how strict the recommendations are. Lower threshold gives more results but less accurate, while higher threshold gives fewer but more accurate results. The evaluation shows reasonable precision and recall, meaning the system is useful but can be improved further by including user ratings history.**